# Hungry Lion MI Analyst Excel assessment

In [1]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt

In [4]:
data = pd.read_excel('./mi_analyst.xlsx',sheet_name=['data'])['data']

In [5]:
data.head(5)

,Month,country,region,store_name,date,Actual_Sales_TY,Transactions,Daily_Target,Actual_Sales_LY,Days_lost,Target Diff,% To Target,sales_forecast,% to Forecast
0,Sept 2022,NAMIBIA,NAMIBIA SOUTH,WALVISBAY,1 Sept 2022,46597.7375,597,43824,33988.8709,0.0,2773.7375,1.063293,43804,1.062809
1,Sept 2022,NAMIBIA,NAMIBIA SOUTH,WINDHOEK TOWN SQ,1 Sept 2022,46528.7810,826,44674,31774.0875,0.0,1854.7810,1.041518,44664,1.040802
2,Sept 2022,NAMIBIA,NAMIBIA SOUTH,MARIENTAL,1 Sept 2022,17987.3045,389,19249,16817.1291,0.0,-1261.6955,0.934454,19232,0.934428
3,Sept 2022,NAMIBIA,NAMIBIA NORTH,OSHAKATI,1 Sept 2022,49003.3006,744,43838,33918.4356,0.0,5165.3006,1.117827,43854,1.116401
4,Sept 2022,NAMIBIA,NAMIBIA NORTH,RUNDU MALL,1 Sept 2022,35641.1309,551,39945,34493.5848,0.0,-4303.8691,0.892255,39965,0.890996


### Question 1
Which store has the highest absolute difference between the Actual_Sales_TY and sales_forecast fields for September 2022?

In [54]:
store_name = None
best_delta = - np.inf
unique_names = set(data['store_name'])
for name in unique_names:
    sub_data = data[(data['Month']=='Sept 2022') & (data['store_name'] == name)]
    delta = abs(sub_data['Actual_Sales_TY'].sum() - sub_data['sales_forecast'].sum())
    if delta > best_delta:
        store_name = name
        best_delta = delta

In [55]:
print('The store with the largest difference of {0} in September 2022 is {1}.'.format(best_delta,store_name))

The store with the largest difference of 426405.51609999966 in September 2022 is MARK PARK.


### Question 2
For the month of October 2022 which region has the lowest value for year on year Actual_Sales growth?

In [42]:
regions = set(data['region'])
worst_region = None
worst_growth = np.inf
for region in regions:
    sub_data = data[(data['Month'] == 'Oct 2022') & (data['region'] == region)]
    region_growth = sub_data['Actual_Sales_TY'].sum() - sub_data['Actual_Sales_LY'].sum()
    if region_growth < worst_growth:
        worst_region = region
        worst_growth = region_growth
print('The region with the worst total growth of {0} is {1}'.format(worst_growth,worst_region))  

The region with the worst total growth of -11544.217499999795 is WCAPE GARDENROUTE


#### NOTE: 
I assumed growth to be a difference between sales and not a rate. The rate solution follows:

In [57]:
regions = set(data['region'])
worst_region = None
worst_growth = np.inf
for region in regions:
    sub_data = data[(data['Month'] == 'Oct 2022') & (data['region'] == region)]
    region_growth = sub_data['Actual_Sales_TY'].sum() - sub_data['Actual_Sales_LY'].sum()
    region_growth /= sub_data['Actual_Sales_LY'].sum()  # converts it into a growth rate
    region_growth *= 100 # converts it into a percentage rate of growth
    if region_growth < worst_growth:
        worst_region = region
        worst_growth = region_growth
print('The region with the worst total growth of {0}% is {1}'.format(worst_growth,worst_region)) 

The region with the worst total growth of -0.6567028133855118% is WCAPE GARDENROUTE


When using percentage growth, the to answers remain the same. This is coincidence, however, as the two methods are not mathematically equivalent such that this outcome cannot be guaranteed. It is good to report both figures.

### Question 3
Which Store has the highest overall Actual_Sales contribution to Actual_Sales_TY for the two months?

In [37]:
unique_names = set(data['store_name'])
best_contribution = - np.inf
store_name = None
for name in unique_names:
    sub_data = data[(data['store_name']==name)]
    total = sub_data['Actual_Sales_TY'].sum()
    if total > best_contribution:
        store_name = name
        best_contribution = total
print('The store with the best contribution of {0} is {1}'.format(best_contribution,store_name))

The store with the best contribution of 6058551.6921999585 is KABWE


#### NOTE: 
I have assumed contribution to be an absolute value and not a percentage contribution in the above calculation. The two approaches are mathematically equivalent though. However, a percentage would be nicer for presentation purposes.

### Question 4
Assuming the ATV (Average Transaction Value) can be calculated as Actual_Sales_TY / Transactions. Which store had the largest month on month difference in ATV Between September and October?

In [61]:
best_store = None
best_atv_diff = - np.inf
unique_names = set(data['store_name'])
for name in unique_names:
    october = data[(data['store_name'] == name) & (data['Month'] == 'Oct 2022')]
    sept = data[(data['store_name'] == name) & (data['Month'] == 'Sept 2022')]
    try:
        oct_atv = october['Actual_Sales_TY'].sum()/october['Transactions'].sum()
    except KeyError:
        continue
    try:
        sept_atv = sept['Actual_Sales_TY'].sum()/october['Transactions'].sum()
    except KeyError:
        continue
    atv_diff = abs(oct_atv - sept_atv)
    #atv_diff = oct_atv - sept_atv
    if atv_diff > best_atv_diff:
        best_store = name
        best_atv_diff = atv_diff
print('The store with the largest month-on-month atv difference is {0} with a value of {1}'.format(best_store,best_atv_diff))

The store with the largest month-on-month atv difference is BOITEKONG with a value of 144.78562391906326


#### NOTE:
In the above calculation, I calculated the store ATV for the month by first summing up 'Actual_Sales_TY' for each month and then dividing it by the total 'Transactions' for that month. This is as opposed to calculating the ATV for each day of the month for a store and then summing that up. Such a calculation would have a different interpretation.

#### NOTE:
I have calculated the month-on-month difference as ``` atv_diff = abs(oct_atv - sept_atv)``` which goes opposed to ```atv_diff = oct_atv - sept_atv```. This means I only looked at the magnitude of the difference between the two months and not the sign/direction. In other, words I did not look at whether October was less than September which would result in a negative ATV. If we are interested in both sign and magnitude then ```HEBRON MALL``` is the answer as opposed to ```BOITEKONG```.